## Model 0

In [3]:
#### import pandas as pd
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
import igraph as ig
import geopy.distance
import folium
from math import radians, cos, sin, asin, sqrt
import networkx as nx
from geopy.distance import geodesic
from itertools import product
 
# Load your dataset and remove NA values
customer_orders_points_07052024 = pd.read_csv("E:/Datasets/Soudal data/Customer_orders_points_07052024.csv")
 
customer_id = customer_orders_points_07052024['POSTCODE'].dropna()
order_quantity = customer_orders_points_07052024['Quantity_Delivered'].dropna()
latitude = customer_orders_points_07052024['y'].dropna()
latitude = latitude[(latitude <= 90) & (latitude >= -90)]
 
longitude = customer_orders_points_07052024['x'].dropna()
longitude = longitude[(longitude <= 90) & (longitude >= -90)]
 
min_length = min(len(longitude), len(latitude))  # min(len(longitude), len(latitude)) or any other criteria
 
# Standardize all vector lengths
customer_id = customer_id.head(min_length)
order_quantity = order_quantity.head(min_length)
latitude = latitude.head(min_length)
longitude = longitude.head(min_length)
 
# Create DataFrame from customer order data
customer_data = pd.DataFrame({
    'Customer': customer_id,
    'Latitude': latitude,
    'Longitude': longitude,
    'Order_quantity': order_quantity
})
 
# Create grouped dataset
grouped_data = customer_data.groupby(['Latitude', 'Longitude', 'Order_quantity']).agg(Num_Customers=('Customer', 'nunique')).reset_index()
 
# Combine orders per location
combined_orders = grouped_data.groupby(['Latitude', 'Longitude']).agg(
    Total_Order_Quantity=('Order_quantity', 'sum'),
    Total_Num_Customers=('Num_Customers', 'sum')
).reset_index()
 
# Perform clustering using DBSCAN on combined_orders dataset based on geoposition only
eps = 0.62  # Adjust epsilon (neighborhood distance) as needed
minPts = 39  # Adjust minPts (minimum number of points in a cluster) as needed
 
dbscan = DBSCAN(eps=eps, min_samples=minPts, metric='euclidean')
dbscan_result = dbscan.fit_predict(combined_orders[['Longitude', 'Latitude']])
 
# Assign cluster labels to data points
combined_orders['Cluster'] = dbscan_result
 
# Calculate cluster centers
cluster_centers_df = combined_orders.groupby('Cluster').agg(
    Longitude=('Longitude', 'mean'),
    Latitude=('Latitude', 'mean'),
    Total_Order_Quantity=('Total_Order_Quantity', 'sum'),
    Count=('Latitude', 'size')
).reset_index()
cluster_centers_df['ID'] = ['Cluster_' + str(i + 1) for i in range(len(cluster_centers_df))]
 
customer_nodes = cluster_centers_df[['ID', 'Total_Order_Quantity', 'Latitude', 'Longitude']]
 
# Create a dataframe
df = pd.DataFrame({
    'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'Country': ["Portugal", "Spain", "Italy", "France", "Belgium", "Latvia", "Poland", "Portugal", "Slovenia", "Spain", "Turkey", "Belgium"],
    'City': ["Lissabon", "Alovera", "Cologno Monzese", "Blyes", "Turnhout", "Dobele", "Pionki", "Lissabon", "Srpenica", "Alovera", "Tuzla, Istanbul", "Pelt"],
    'Plant_type': ["Receiving plant", "Receiving plant", "Receiving plant", "Receiving plant", "Producing plant", "Producing plant", "Producing plant", "Producing plant", "Producing plant", "Producing plant", "Producing plant", "Producing plant"],
    'ZIP_code': ["2710-207", "19208", "20093", "1150", "2300", "3701", "26-670", "2710-207", "5224", "19208", "34956", "3900"],
    'Receiving_plant': [2200, 1900, 2000, 1800, None, None, None, None, None, None, None, None],
    'X_coordinate': [-9.369216, -3.243162431, 9.281575673, 5.261173337, 4.928540898, 23.30237845, 21.45811508, -9.369216, 13.5166732, -3.243162431, 29.36691489, 5.3966],
    'Y_coordinate': [38.757301, 40.57675934, 45.54113003, 45.83747369, 51.29525785, 56.62518423, 51.46902482, 38.757301, 46.28409136, 40.57675934, 40.8858807, 51.1913],
    'Geometry': ["POINT (-9.36922 38.75730)", "POINT (-3.24316 40.57676)", "POINT (9.28158 45.54113)", "POINT (5.26117 45.83747)", "POINT (4.92854 51.29526)", "POINT (23.30238 56.62518)", "POINT (21.45812 51.46902)", "POINT (-9.36922 38.75730)", "POINT (13.51667 46.28409)", "POINT (-3.24316 40.57676)", "POINT (29.36691 40.88588)", "POINT (5.3966,51.1913)"]
})
 
# Load in the interplant orders
Interplant_Orders_2023_2 = pd.read_csv("E:/Datasets/Soudal data/Interplant_Orders_2023_2.csv")
 
# Assign an easier name to the variable
Order_Ref = Interplant_Orders_2023_2
 
# Split the aggregated data into producing plants and receiving plants
producing_plants = df[df['Plant_type'] == "Producing plant"]
receiving_plants = df[df['Plant_type'] == "Receiving plant"]
 
# Create a new dataframe with the desired structure
new_df = pd.DataFrame({
    'ID': df['City'] + " plant " + df['ID'].astype(str),
    'Total_Quantity': df['Receiving_plant'],
    'Latitude': df['Y_coordinate'],
    'Longitude': df['X_coordinate']
})
 
# Convert Total_Quantity to integer
new_df['Total_Quantity'] = new_df['Total_Quantity'].astype(pd.Int64Dtype())
 
# # Group the data by warehouse and calculate both total capacity and frequency
# warehouse_capacity = Order_Ref.groupby('Producing_Plant').agg(
#     Total_Capacity=('Quantity_Delivered', 'sum'),
#     Frequency=('Producing_Plant', 'count')
# ).reset_index()
 
# # Create a lookup table mapping country names to country codes
# country_lookup_producing = pd.DataFrame({
#     'Country': ["Belgium", "Latvia", "Poland", "Portugal", "Slovenia", "Spain", "Turkey"],
#     'Country_Code': ["BE", "LV", "PL", "PT", "SI", "ES", "TR"]
# })
 
# # Create a lookup table mapping country names to country codes
# country_lookup_receiving = pd.DataFrame({
#     'Country': ["Lissabon", "Alovera", "Cologno Monzese", "Byles"],
#     'Country_Code': ["PT", "ES", "IT", "FR"]
# })
 
# # Replace the country names with country codes in the producing dataframe
# producing_plants['Country'] = producing_plants['Country'].map(dict(zip(country_lookup_producing['Country'], country_lookup_producing['Country_Code'])))
 
# # Replace the country names with country codes in the receiving dataframe
# receiving_plants['Country'] = ["PT", "ES", "IT", "FR"]
 
# # Grouping by Producing_Plant and Receiving_Plant, and summarizing the Quantity_Delivered and Order_Ref
# flow_data = Order_Ref.groupby(['Producing_Plant', 'Receiving_Plant']).agg(
#     Total_Quantity=('Quantity_Delivered', 'sum'),
#     Frequency=('Producing_Plant', 'count')
# ).reset_index()
 
# # Arranging the data by frequency in descending order
# flow_data = flow_data.sort_values(by=['Producing_Plant'])
 
# # Assigning country codes and coordinates
# flow_data['Country'] = ["BE", "PL", "GB", "BE", "BE", "NL", "DE", "PL", "TR", "SI", "PL", "BE", "NL"]
# flow_data['X_coordinate'] = [51.3377, 52.3753, 52.6043, 51.1913, 51.1055, 51.2518, 51.0625, 51.4583, 40.6295, 46.2914, 51.4583, 51.1055, 51.5064]
# flow_data['Y_coordinate'] = [4.9345, 20.6860, 1.6485, 5.3966, 3.8589, 5.6885, 6.9419, 21.4481, 30.6428, 13.4568, 21.4481, 3.8589, 4.2572]
 
# flow_data.columns = ["Producing_Plant", "Receiving_Plant", "Total Quantity", "Frequency", "Country", "Latitude", "Longitude"]
 
# # Assigning IDs
# flow_data['ID'] = ["Turnhout plant 1000", "Dąbrówka plant 1030", "Great_Britain plant 1100", "Pelt plant 1400", "Gent plant 1500",
#                    "Weert plant 1600", "Leverkussen plant 1700", "Pionki plant 2400", "Turkey Plant 5200", "Srpenica plant 5600",
#                    "Pionki plant 5900", "Gent plant 6700", "Bergen_op_zoom plant 8700"]
 
# # Selecting specific columns
# logistical_flow_nodes = flow_data[['ID', 'Total Quantity', 'Latitude', 'Longitude']]
logistical_nodes = new_df[['ID', 'Latitude', 'Longitude']]
 
# Given plant dataset
plant_dataset = logistical_nodes
 
# Given customer dataset
customer_dataset = pd.DataFrame({
    'ID': cluster_centers_df['ID'],
    'Longitude': cluster_centers_df['Longitude'],
    'Latitude': cluster_centers_df['Latitude']
})
 
# Create the final combined dataset
combined_dataset = pd.concat([new_df[['ID', 'Latitude', 'Longitude']], cluster_centers_df[['ID', 'Latitude', 'Longitude']]], ignore_index=True)

# Adjust index to start from 1
combined_dataset.index = combined_dataset.index + 1

# Calculate distances between each pair of points
def calculate_distance(point1, point2):
    return geodesic(point1, point2).kilometers

distances = np.zeros((len(combined_dataset), len(combined_dataset)))

for i, j in product(range(len(combined_dataset)), repeat=2):
    distances[i, j] = calculate_distance(
        (combined_dataset['Latitude'].iloc[i], combined_dataset['Longitude'].iloc[i]),
        (combined_dataset['Latitude'].iloc[j], combined_dataset['Longitude'].iloc[j])
    )

# Create a graph from the distance matrix
g = nx.from_numpy_array(distances, create_using=nx.Graph)

def create_map(combined_dataset):
    # Create a folium map centered around the average latitude and longitude
    avg_lat = combined_dataset['Latitude'].mean()
    avg_lon = combined_dataset['Longitude'].mean()
    folium_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=5)

    # Add markers to the map
    for _, row in combined_dataset.iterrows():
        if 'Cluster' in row['ID']:
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=f"Cluster: {row['ID']}",
                icon=folium.Icon(color='blue'),
                tooltip=row['ID']
            ).add_to(folium_map)
        else:
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=f"Warehouse: {row['ID']}",
                icon=folium.Icon(color='red'),
                tooltip=row['ID']
            ).add_to(folium_map)
    
    # Add legend
    legend_html = """
    <div style="position: fixed; bottom: 50px; right: 50px; z-index:9999; background-color:white; border:2px solid grey; padding: 10px;">
    <h4>Legend</h4>
    <p><span style="color:blue">&#9632;</span> Customer Clusters</p>
    <p><span style="color:red">&#9632;</span> Warehouses</p>
    </div>
    """
    folium_map.get_root().html.add_child(folium.Element(legend_html))

    # Display the map
    display(folium_map)

# Display the map
create_map(combined_dataset)

# Function to select specified nodes manually
def select_nodes():
    selected_nodes = input("Enter the row numbers of the nodes you want to include, separated by commas: ")
    selected_nodes = list(map(int, selected_nodes.split(',')))
    return selected_nodes

# Display the map before selecting nodes
print(combined_dataset) 

# Specify nodes for the objective function
specified_nodes = select_nodes()  # The script stops here and waits for user input

def objective_function_dijkstra(coords, specified_nodes, combined_dataset, g):
    # Determine the index for the new node
    new_node_index = len(combined_dataset)
    # Create a temporary graph to add the new node and edges
    temp_graph = g.copy()
    temp_graph.add_node(new_node_index)
    # Calculate the total distance from the new warehouse to each specified node
    total_distance = 0
    for i in specified_nodes:
        distance = calculate_distance(
            (coords['Latitude'], coords['Longitude']),
            (combined_dataset['Latitude'].iloc[i], combined_dataset['Longitude'].iloc[i])
        )
        total_distance += distance
    return total_distance

# Define grid search ranges
lon_range = np.linspace(combined_dataset['Longitude'].min(), combined_dataset['Longitude'].max(), num=100)
lat_range = np.linspace(combined_dataset['Latitude'].min(), combined_dataset['Latitude'].max(), num=100)

grid_search = pd.DataFrame(list(product(lon_range, lat_range)), columns=['Longitude', 'Latitude'])

# Calculate objective function values for each grid point
objective_values = grid_search.apply(lambda row: objective_function_dijkstra(row, specified_nodes, combined_dataset, g), axis=1)

# Find the coordinates of the new position that minimize the objective function
optimal_index = np.argmin(objective_values)
optimal_coords = grid_search.iloc[optimal_index]

# Coordinates of the new position that minimize the total distance
new_position_longitude = optimal_coords['Longitude']
new_position_latitude = optimal_coords['Latitude']

# Sample data
# customers = customer_nodes
# warehouses = logistical_nodes

new_warehouse = pd.DataFrame({
    'ID': ['New warehouse'],
    'Latitude': [new_position_latitude],
    'Longitude': [new_position_longitude]
})
 
# Create a folium map
m = folium.Map(location=[new_position_latitude, new_position_longitude], zoom_start=10)
 
# Add circle markers for customers
for idx, row in customer_dataset.iterrows():
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']], radius=5, color='blue', fill=True, fill_color='blue', fill_opacity=0.6, tooltip=row['ID']).add_to(m)
 
# Add circle markers for existing warehouses
for idx, row in plant_dataset.iterrows():
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']], radius=5, color='red', fill=True, fill_color='red', fill_opacity=0.6, tooltip=row['ID']).add_to(m)
 
# Add circle marker for new warehouse
folium.CircleMarker(location=[new_position_latitude, new_position_longitude], radius=5, color='green', fill=True, fill_color='green', fill_opacity=0.6, tooltip='New Warehouse').add_to(m)
 
# Add layers control
folium.LayerControl().add_to(m)
 
# Add title and legend
title_html = '<h3 style="text-align: center;">Scenario 1</h3>'
legend_html = """
<div id="legend" class="info">
<h4>Legend</h4>
<div class="legend-item"><i style="background: blue"></i>Central Customer Clusters</div>
<div class="legend-item"><i style="background: red"></i>Existing Warehouses</div>
<div class="legend-item"><i style="background: green"></i>Search area for new location</div>
</div>
"""
 
m.get_root().html.add_child(folium.Element(title_html), name='title')
m.get_root().html.add_child(folium.Element(legend_html), name='legend')

print(new_warehouse) 

# Display the map
m

C:\Users\Marnik\AppData\Local\Temp\ipykernel_21920\1981769176.py:15: DtypeWarning: Columns (6,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  customer_orders_points_07052024 = pd.read_csv("E:/Datasets/Soudal data/Customer_orders_points_07052024.csv")


                          ID   Latitude  Longitude
1           Lissabon plant 1  38.757301  -9.369216
2            Alovera plant 2  40.576759  -3.243162
3    Cologno Monzese plant 3  45.541130   9.281576
4              Blyes plant 4  45.837474   5.261173
5           Turnhout plant 5  51.295258   4.928541
6             Dobele plant 6  56.625184  23.302378
7             Pionki plant 7  51.469025  21.458115
8           Lissabon plant 8  38.757301  -9.369216
9           Srpenica plant 9  46.284091  13.516673
10          Alovera plant 10  40.576759  -3.243162
11  Tuzla, Istanbul plant 11  40.885881  29.366915
12             Pelt plant 12  51.191300   5.396600
13                 Cluster_1  43.612755   0.061978
14                 Cluster_2  37.184237  -3.841719
15                 Cluster_3  38.789168  -0.594103
16                 Cluster_4  40.407623  -3.716632
17                 Cluster_5  41.609845   2.165340
18                 Cluster_6  42.902094  -8.478456
19                 Cluster_7  4

Enter the row numbers of the nodes you want to include, separated by commas:  1,2,3


              ID   Latitude  Longitude
0  New warehouse  45.628285   5.499198
